# **import libraries**

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import keras
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import  concatenate
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
# make the results reproducible:
import random
random.seed(0)
import tensorflow as tf
np.random.seed(0)
tensorflow.random.set_seed(0)

# **mount google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **fetch data from google drive**

In [ ]:
!cp /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/anti_spoofing_data.npz /content
!cp /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/anti_spoofing_data_part2.npz /content

# **load data**

In [ ]:
#load data part1
anti_spoofing_data_part1 = np.load('anti_spoofing_data.npz')                      # load the npz file part 1 (40,000 image samples)
X1, y1 = anti_spoofing_data_part1['arr_0'], anti_spoofing_data_part1['arr_1']     # store the images and their corresponding labels into X1 and y1, respectively
del anti_spoofing_data_part1                                                      # delete this variable to free up the memory
#load data part2
anti_spoofing_data_part2 = np.load('anti_spoofing_data_part2.npz')                # load the npz file part 2 (40,000 image samples)
X2, y2 = anti_spoofing_data_part2['arr_0'], anti_spoofing_data_part2['arr_1']
del anti_spoofing_data_part2
data_all=np.concatenate([X1,X2])                                                  # concatenate image samples from part 1 and part 2
label_all=np.concatenate([y1,y2])                                                 # concatenate image labels from part 1 and part 2
# delete all the unused variables to free up the memory
del X1
del X2
del y1
del y2

In [ ]:
# garbage collector
import gc 
gc.collect() 

8375

# **display the image samples**

In [ ]:
print("data shape:",data_all.shape)
print("labels shape",label_all.shape)

In [ ]:
plt.figure(figsize=(224,224))
images=[0,28,1,15060,9000,27,15061,29,500,1024]
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(data_all[images[i]])
plt.show()

In [ ]:
# Due to the limitation of resources, 50k from 80k number of samples were used in training the models.
data_all=data_all[0:50000]
label_all=label_all[0:50000]

In [ ]:
print("data shape:",data_all.shape)
print("labels shape",label_all.shape)

# **split data into train, validation, and test sets with a ratio of 80:10:10**

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_valid, y_train, y_valid = train_test_split(data_all, label_all, test_size=0.2, random_state=42,shuffle=True)


In [ ]:
del data_all
del label_all

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42,shuffle=True)
print(f'Training dataset size of X_train: {len(X_train)}')
print(f'Testing dataset size of X_test: {len(X_test)}')
print(f'Validation dataset size of X_valid: {len(X_valid)}')
print(f'Testing dataset size of y_train: {len(y_train)}')
print(f'Testing dataset size of y_test: {len(y_test)}')
print(f'Testing dataset size of y_valid: {len(y_valid)}')

In [ ]:
import gc 
gc.collect() 

In [ ]:
width=224
height=224
channel=3

In [ ]:
my_epochs=50
my_batch_size=128

# **create VGG16 model**

In [ ]:
VGG_base_model=tf.keras.applications.VGG16(weights="imagenet",include_top=False,input_shape =(width, height,channel))  # remove three FC layers from VGG-16 by setting include_top as False
VGG_base_model.summary ()

for layer in VGG_base_model.layers:           
    layer.trainable = False                              # fix the parameters of convolutional bas in VGG-16 (they will not be changed during training the model on our data)

#get weights and features from base model    
VGG_features = VGG_base_model.output


# **attach the extracted features from VGG-16 to our own classifier**

In [ ]:
headModel = VGG_base_model.output                                    # get the features from VGG-16
headModel = Flatten(name="flatten")(headModel)                       # flatten the features
headModel = Dense(300, activation="relu")(headModel)                 # the first FC layer with 300 neurons
headModel = Dropout(.5)(headModel)                                   # droput layer with a rate of 0.5 
headModel = Dense(200, activation="relu")(headModel)                 # the second FC layer with 200 neurons
headModel = Dropout(.5)(headModel)                                   # droput layer with a rate of 0.5 
headModel = Dense(100, activation="relu")(headModel)                  # the third FC layer with 100 neurons
headModel = Dense(1, activation="sigmoid")(headModel)                # output neuron followed by a Sigmoid activation function
vgg_model = Model(inputs=VGG_base_model.input, outputs=headModel)    # group all the layers into vgg_model

# **compile and plot model created using VGG model**

In [ ]:
vgg_model.compile(optimizer='Adam', loss=tensorflow.keras.losses.binary_crossentropy,metrics="accuracy" )    # compile the model with specified optimizer, loss, and metrics
keras.utils.vis_utils.plot_model(vgg_model,dpi=100)                                                          # plot the model


# **train VGG model**

In [ ]:
vgg_hist=vgg_model.fit(X_train,y_train, epochs=my_epochs,batch_size=my_batch_size,validation_data=(X_valid,y_valid))

# **save model**

In [ ]:
from keras.models import load_model

vgg_model.save('vgg_model.h5')  # creates a HDF5 file to save model
 

In [ ]:
!cp vgg_model.h5 /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/    # save the model in Google Drive for future use

# **load vgg trained model**
The two cells below are useful when the model has already been trained and saved in Google Drive, and you can load the model to test it or derive new plots.

In [ ]:
!cp  /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/vgg_model.h5 /content/

In [ ]:
from keras.models import load_model

# returns a compiled model identical to the previous one
vgg_model = load_model('/content/vgg_model.h5')

# **plot training history: loss and accuracy**

In [ ]:

acc = vgg_hist.history['accuracy']
loss = vgg_hist.history['loss']
val_acc=vgg_hist.history['val_accuracy']
val_loss=vgg_hist.history['val_loss']

plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), loss, label='training loss')
plt.plot(range(my_epochs), val_loss, label='validation loss')
plt.legend(loc='upper right')
plt.title('Training loss ')
plt.show()
plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), acc, label='training accuracy' )
plt.plot(range(my_epochs), val_acc, label='validation accuracy' )
plt.legend(loc='upper right')
plt.title('Training accuracy')
plt.show()


# **test model**

In [ ]:
y_pred_vgg=vgg_model.predict(X_test,verbose=0)
y_pred_vgg=(y_pred_vgg>0.5).astype("int32")      # set a threshold of 0.5 for the sigmoid-activated neuron: if its value is above 0.5, the predicted label is 1, otherwise 0

# **Plot confusion matrix for the results of classification**

In [ ]:
Con_matrix=confusion_matrix(y_test, y_pred_vgg)
fig, ax = plot_confusion_matrix(conf_mat=Con_matrix,
                            show_absolute=True,
                            show_normed=False,
                            colorbar=True)

ax.set_title('confusion_matrix of VGG ')
plt.show()

# **create ResNet50 model**

In [ ]:

ResNet_base_model=tf.keras.applications.ResNet50(weights="imagenet",include_top=False,input_shape =(width,height,channel))
ResNet_base_model.summary ()

for layer in ResNet_base_model.layers:
    layer.trainable = False

#get weights and features from base model    
ResNet_features = ResNet_base_model.output


# **use pretrained ResNet**

In [ ]:
headModel = ResNet_base_model.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(300, activation="relu")(headModel)
headModel = Dropout(.5)(headModel)
headModel = Dense(200, activation="relu")(headModel)
headModel = Dropout(.5)(headModel)
headModel = Dense(100, activation="relu")(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
resnet_model = Model(inputs=ResNet_base_model.input, outputs=headModel)

# **compile and plot model created using ResnNet model**

In [ ]:
resnet_model.compile(optimizer='Adam', loss=tensorflow.keras.losses.binary_crossentropy,metrics="accuracy" )
import keras
keras.utils.vis_utils.plot_model(resnet_model,dpi=150)


# **train ResNet model**

In [ ]:
import gc
gc.collect()

In [ ]:
resnet_hist=resnet_model.fit(X_train,y_train, epochs=my_epochs,batch_size=my_batch_size,validation_data=(X_valid,y_valid))

# **save model**

In [ ]:
from keras.models import load_model

resnet_model.save('resnet_model.h5')  # creates a HDF5 file to save model
 

In [ ]:
!cp resnet_model.h5 /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/

# **load resNet trained model**

In [ ]:
!cp  /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/resnet_model.h5 /content/

In [ ]:
from keras.models import load_model

# returns a compiled model identical to the previous one
resnet_model = load_model('/content/resnet_model.h5')

# **plot loss and accuracy**

In [ ]:

acc = resnet_hist.history['accuracy']
loss = resnet_hist.history['loss']
val_acc=resnet_hist.history['val_accuracy']
val_loss=resnet_hist.history['val_loss']

plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), loss, label='training loss')
plt.plot(range(my_epochs), val_loss, label='validation loss')
plt.legend(loc='upper right')
plt.title('Training loss ')
plt.show()
plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), acc, label='training accuracy' )
plt.plot(range(my_epochs), val_acc, label='validation accuracy' )
plt.legend(loc='upper right')
plt.title('Training accuracy')
plt.show()


# **test model**

In [ ]:
y_pred_resnet=resnet_model.predict(X_test,verbose=0)
y_pred_resnet=(y_pred_resnet>0.5).astype("int32")
# y_pred_vgg[1:10]

# **Plot confusion matrix for the results of classification**

In [ ]:
Con_matrix=confusion_matrix(y_test, y_pred_resnet)
fig, ax = plot_confusion_matrix(conf_mat=Con_matrix,
                            show_absolute=True,
                            show_normed=False,
                            colorbar=True)

ax.set_title('confusion_matrix of ResNet ')
plt.show()


# **create Xception model**

In [ ]:
xception_base_model=tf.keras.applications.Xception(weights="imagenet",include_top=False,input_shape =(width,height,channel))
xception_base_model.summary ()

for layer in xception_base_model.layers:
    layer.trainable = False

#get weights and features from base model    
xception_features = xception_base_model.output


# **use pretrained Xception**

In [ ]:
#get weights and features from base model    
headModel = xception_base_model.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(300, activation="relu")(headModel)
headModel = Dropout(.2)(headModel)
headModel = Dense(200, activation="relu")(headModel)
headModel = Dropout(.2)(headModel)
headModel = Dense(100, activation="relu")(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)
xception_model = Model(inputs=xception_base_model.input, outputs=headModel)

# **compile and plot model created using Xception model**

In [ ]:
xception_model.compile(optimizer='Adam', loss=tensorflow.keras.losses.binary_crossentropy,metrics="accuracy" )
import keras
keras.utils.vis_utils.plot_model(xception_model,dpi=100)


# **train Xception model**

In [ ]:
xception_hist=xception_model.fit(X_train,y_train, epochs=my_epochs,batch_size=my_batch_size,validation_data=(X_valid,y_valid))

# **save model**

In [ ]:
from keras.models import load_model

xception_model.save('xception_model.h5')  # creates a HDF5 file to save model
 

In [ ]:
!cp xception_model.h5 /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/

# **load trained Xception model**

In [ ]:
!cp  /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/xception_model.h5 /content/

In [ ]:
from keras.models import load_model

# returns a compiled model identical to the previous one
xception_model = load_model('/content/xception_model.h5')

# **plot loss and accuracy**

In [ ]:

acc = xception_hist.history['accuracy']
loss = xception_hist.history['loss']
val_acc=xception_hist.history['val_accuracy']
val_loss=xception_hist.history['val_loss']

plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), loss, label='training loss')
plt.plot(range(my_epochs), val_loss, label='validation loss')
plt.legend(loc='upper right')
plt.title('Training loss ')
plt.show()
plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), acc, label='training accuracy' )
plt.plot(range(my_epochs), val_acc, label='validation accuracy' )
plt.legend(loc='upper right')
plt.title('Training accuracy')
plt.show()


# **test model**

In [ ]:
y_pred_xception=xception_model.predict(X_test,verbose=0)
y_pred_xception=(y_pred_xception>0.5).astype("int32")


# **Plot confusion matrix for the results of classification**

In [ ]:
Con_matrix=confusion_matrix(y_test, y_pred_xception)
fig, ax = plot_confusion_matrix(conf_mat=Con_matrix,
                            show_absolute=True,
                            show_normed=False,
                            colorbar=True)

ax.set_title('confusion_matrix of  Xception ')
plt.show()


# **create hybrid model**

In [ ]:
VGG_features=Flatten(name="VGG_Weights_Features")(VGG_features)

In [ ]:
ResNet_features=Flatten(name="ResNet_Weights_Features")(ResNet_features)

In [ ]:
all_features = concatenate([VGG_features, ResNet_features])

In [ ]:
headModel = Dense(300, activation="relu")(all_features)
headModel = Dropout(.5)(headModel)
headModel = Dense(200, activation="relu")(headModel)
headModel = Dropout(.5)(headModel)
headModel = Dense(100, activation="relu")(headModel)
head_mode_output = Dense(1,activation="sigmoid")(headModel)
hybrid_model = Model(inputs=[VGG_base_model.input,ResNet_base_model.input], outputs=head_mode_output,name="final_output")
hybrid_model.compile(optimizer='Adam', loss=tensorflow.keras.losses.binary_crossentropy,metrics="accuracy" )


# **plot model**

In [ ]:
import keras
keras.utils.vis_utils.plot_model(hybrid_model,dpi=150)


# **train model**

In [ ]:
import gc
gc.collect()

887

In [ ]:
my_batch_size=32
hybrid_model_hist=hybrid_model.fit([X_train,X_train], y_train,
                           validation_data=([X_valid,X_valid], y_valid), epochs=my_epochs,batch_size=my_batch_size)

# **save model**

In [ ]:
from keras.models import load_model

hybrid_model.save('hybrid_model.h5')  # creates a HDF5 file to save model
 

In [ ]:
!cp hybrid_model.h5 /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/

# **load trained hybrid model**

In [ ]:
!cp  /content/drive/MyDrive/Face_Anti_Spoofing_Dataset/hybrid_model_d2.h5 /content/

In [ ]:
from keras.models import load_model

# returns a compiled model identical to the previous one
hybrid_model = load_model('/content/hybrid_model_d5.h5')

# **plot loss and accuracy**

In [ ]:
acc = hybrid_model_hist.history['accuracy']
loss = hybrid_model_hist.history['loss']
val_acc=hybrid_model_hist.history['val_accuracy']
val_loss=hybrid_model_hist.history['val_loss']

plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), loss, label='training loss')
plt.plot(range(my_epochs), val_loss, label='validation loss')
plt.legend(loc='upper right')
plt.title('Training loss ')
plt.show()
plt.figure(figsize=(10,8))
plt.plot(range(my_epochs), acc, label='training accuracy' )
plt.plot(range(my_epochs), val_acc, label='validation accuracy' )
plt.legend(loc='upper right')
plt.title('Training accuracy')
plt.show()


# **test model**

In [ ]:
y_pred_hybrid=hybrid_model.predict([X_test,X_test],verbose=0)
y_pred_hybrid=(y_pred_hybrid>0.5).astype("int32")

# **Plot confusion matrix for the results of classification**

In [ ]:
Con_matrix=confusion_matrix(y_test, y_pred_hybrid)
fig, ax = plot_confusion_matrix(conf_mat=Con_matrix,
                            show_absolute=True,
                            show_normed=False,
                            colorbar=True)

ax.set_title('confusion_matrix of Hybrid model ')
plt.show()
